In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)
import time

# A. Query Data

In [2]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
host = '10.220.50.121'
port = '1661'
service_name = 'ANALYTIC'

In [3]:
connection_string = f'oracle+oracledb://{username}:{password}@{host}:{port}/?service_name={service_name}'

In [4]:
# create engine
engine = create_engine(connection_string)

In [5]:
# create list of all dates
allDates = ['20241031','20240930','20240829','20240731','20240628','20240531',
            '20240430','20240329','20240229','20240131','20231229','20231130']

In [6]:
# open sql file
with open("CC Pull.sql", 'r') as file:
    query_template = file.read()

In [7]:
# print(query_template.format(dt='20241031'))

In [8]:
allMLs = []
for date in allDates:
    try:
        # print in case of errors
        print(f'Running {date}')

        # open and read the file as a single buffer
        sqlQuery = query_template.format(dt=date)

        # run query and store data in df
        print('SQL run')
        with engine.connect() as connection:
            ML = pd.read_sql(sqlQuery, connection) 
        print(f'{date} queried successfully')

        # change column names into UPPER CASE
        ML.columns = [x.upper() for x in ML.columns]
    except Exception as e:
        print(e)
        ML = pd.DataFrame()
    finally:
        allMLs.append(ML)

        time.sleep(5)

Running 20241031
SQL run
20241031 queried successfully
Running 20240930
SQL run
20240930 queried successfully
Running 20240829
SQL run
20240829 queried successfully
Running 20240731
SQL run
20240731 queried successfully
Running 20240628
SQL run
20240628 queried successfully
Running 20240531
SQL run
20240531 queried successfully
Running 20240430
SQL run
20240430 queried successfully
Running 20240329
SQL run
20240329 queried successfully
Running 20240229
SQL run
20240229 queried successfully
Running 20240131
SQL run
20240131 queried successfully
Running 20231229
SQL run
20231229 queried successfully
Running 20231130
SQL run
20231130 queried successfully


In [11]:
allMLs[1]

,BASE_DT,AGREE_ID,GCIF_NO,CIF_NO,NOTE_NO,PRD_TP,PRD_NM,PRD_SEGMENT,ORG_LMT_AMT,CURR_LMT_AMT,BAL,ORIG_INT_RT,CURR_INT_RT,OTH_CURR_INT_RT,CLCT_RTNG_FCL,ALLOW_PCT,ALLOW_PCT_ADD,RESTRUCT_CD,PASTDUE_SINCE_DT,PASTDUE_DAYS,WRITE_OFF_AMT,WRITE_OFF_DT,WRITE_OFF_YN,CC_STATUS,GCIF_NO,SEGMENT,SEGMENT_FUNDING,CUST_NM,OPEN_DATE,LAST_ACTIVE_DATE,CUST_TP,GENDER_CD,AGE,EMPLOYMENT_TYPE,MICRO_SEGMENT,CA_BAL,CA_AVG,SA_BAL,SA_AVG,TD_NOA,TD_BAL,TD_AVG,SY_FUNDING_BAL,SY_FUNDING_AVG,FUNDING_NOA,FUNDING_BAL,FUNDING_AVG,NOA_DORMANT,AUM,CC_PLAFOND,CC_BAL,PL_PLAFOND,PLOAN_BAL,TR_PLAFOND,TR_BAL,PB_PLAFOND,PB_BAL,PRK_PLAFOND,PRK_BAL,PPB_PLAFOND,PPB_BAL,BG_PLAFOND,BG_BAL,LC_PLAFOND,LC_BAL,SY_LOAN_PLAFOND,SY_LOAN_BAL,KPR_PLAFOND,KPR_BAL,KPM_PLAFOND,KPM_BAL,COLLECT,LUM,BA_UL_BAL,BA_ALL_BAL,MF_BAL,BONDS_BAL,TRB,M2U_TRX_TOTAL,M2U_TRXAMT_FIN,CRTRX_MTD,CRTRX_AMT_MTD,DBTRX_MTD,DBTRX_AMT_MTD
0,20240930,CP0010014423730005712016,G800008963,1010000540230352,4423730005712016,001,Credit Card,031100,15000000.0,7.848902e+06,7848902.0,0.21,0.21,0.21,00001,0.01,0.0,00008,99991231,0,0.0,None,N,DELINQUENT,G800008963,None,None,PETER CHANDRA,1997-02-27,NaT,INDV,M,53.0,OTHERS,4.Silver Age,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,50000000.0,50728338.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.0,5.072834e+07,0.0,0.0,0.0,0.0,5.072834e+07,0.0,0.0,0.0,0.0,0.0,0.0
1,20240930,CP0010014423730007054110,G200315984,1010005000389119,4423730007054110,001,Credit Card,031100,4000000.0,2.728878e+07,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,DELINQUENT,G200315984,None,None,HARRY WINATA,1992-03-20,NaT,INDV,M,74.0,SELF EMPLOYED,5.Golden Age,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,23000000.0,1267295.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.0,6.267295e+06,0.0,0.0,0.0,0.0,6.267295e+06,0.0,0.0,0.0,0.0,0.0,0.0
2,20240930,CP0010014423730007772018,G500626999,1040021080462209,4423730007772018,001,Credit Card,031100,23000000.0,6.000000e+04,60000.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,DELINQUENT,G500626999,CFS-NONRB-RSME,CFS-NONRB-RSME,ABADI PANGESTU,1991-06-24,2024-09-30,INDV,M,68.0,SELF EMPLOYED,5.Golden Age,0.0,0.0,16619710.05,17068756.75,0.0,0.0,0.0,16619710.05,17068756.75,3.0,16619710.05,17068756.75,0.0,16619710.05,23000000.0,60000.0,0.0,0.0,0.0,0.0,560000000.0,34023129.46,8.298806e+08,8.298806e+08,0.0,0.0,0.0,0.0,0.0,0.0,560000000.0,34023129.46,560000000.0,34023129.46,0.0,0.0,1.0,8.639638e+08,0.0,0.0,0.0,0.0,8.805835e+08,1.0,0.0,6.0,299106038.0,43.0,588301623.0
3,20240930,CP0010014423730008087002,G301797247,1080520000001519,4423730008087002,001,Credit Card,031100,3000000.0,1.321008e+06,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,DELINQUENT,G301797247,None,None,"DWI HERI YATMOKO, DRS",2005-08-08,NaT,INDV,M,66.0,OTHERS,5.Golden Age,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,4000000.0,36977.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.0,3.697700e+04,0.0,0.0,0.0,0.0,3.697700e+04,0.0,0.0,0.0,0.0,0.0,0.0
4,20240930,CP0010014423730011419010,G700015878,1010001100061864,4423730011419010,001,Credit Card,031100,22000000.0,1.533333e+07,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,DELINQUENT,G700015878,None,None,DJOKO MARTADHI H.,1990-04-06,NaT,INDV,M,61.0,OTHERS,5.Golden Age,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,22000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# B. EDA

In [12]:
# concat all MLs into one
MLs = pd.concat(allMLs, ignore_index=True)

C:\Users\MR38804\AppData\Local\Temp\ipykernel_16576\416592903.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  MLs = pd.concat(allMLs, ignore_index=True)


In [13]:
MLs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995100 entries, 0 to 1995099
Data columns (total 84 columns):
 #   Column            Dtype         
---  ------            -----         
 0   BASE_DT           object        
 1   AGREE_ID          object        
 2   GCIF_NO           object        
 3   CIF_NO            object        
 4   NOTE_NO           object        
 5   PRD_TP            object        
 6   PRD_NM            object        
 7   PRD_SEGMENT       object        
 8   ORG_LMT_AMT       float64       
 9   CURR_LMT_AMT      float64       
 10  BAL               float64       
 11  ORIG_INT_RT       float64       
 12  CURR_INT_RT       float64       
 13  OTH_CURR_INT_RT   float64       
 14  CLCT_RTNG_FCL     object        
 15  ALLOW_PCT         float64       
 16  ALLOW_PCT_ADD     float64       
 17  RESTRUCT_CD       object        
 18  PASTDUE_SINCE_DT  object        
 19  PASTDUE_DAYS      object        
 20  WRITE_OFF_AMT     float64       
 21  WRITE_OF

In [14]:
MLs.columns

Index(['BASE_DT', 'AGREE_ID', 'GCIF_NO', 'CIF_NO', 'NOTE_NO', 'PRD_TP',
       'PRD_NM', 'PRD_SEGMENT', 'ORG_LMT_AMT', 'CURR_LMT_AMT', 'BAL',
       'ORIG_INT_RT', 'CURR_INT_RT', 'OTH_CURR_INT_RT', 'CLCT_RTNG_FCL',
       'ALLOW_PCT', 'ALLOW_PCT_ADD', 'RESTRUCT_CD', 'PASTDUE_SINCE_DT',
       'PASTDUE_DAYS', 'WRITE_OFF_AMT', 'WRITE_OFF_DT', 'WRITE_OFF_YN',
       'CC_STATUS', 'GCIF_NO', 'SEGMENT', 'SEGMENT_FUNDING', 'CUST_NM',
       'OPEN_DATE', 'LAST_ACTIVE_DATE', 'CUST_TP', 'GENDER_CD', 'AGE',
       'EMPLOYMENT_TYPE', 'MICRO_SEGMENT', 'CA_BAL', 'CA_AVG', 'SA_BAL',
       'SA_AVG', 'TD_NOA', 'TD_BAL', 'TD_AVG', 'SY_FUNDING_BAL',
       'SY_FUNDING_AVG', 'FUNDING_NOA', 'FUNDING_BAL', 'FUNDING_AVG',
       'NOA_DORMANT', 'AUM', 'CC_PLAFOND', 'CC_BAL', 'PL_PLAFOND', 'PLOAN_BAL',
       'TR_PLAFOND', 'TR_BAL', 'PB_PLAFOND', 'PB_BAL', 'PRK_PLAFOND',
       'PRK_BAL', 'PPB_PLAFOND', 'PPB_BAL', 'BG_PLAFOND', 'BG_BAL',
       'LC_PLAFOND', 'LC_BAL', 'SY_LOAN_PLAFOND', 'SY_LOAN_BAL', 'KPR_PL

In [16]:
MLs = MLs.loc[:, ~MLs.columns.duplicated()].copy()

In [25]:
MLs[['PASTDUE_DAYS','MICRO_SEGMENT','CC_STATUS']].value_counts()

PASTDUE_DAYS  MICRO_SEGMENT         CC_STATUS 
0             3.Mid Lifers          DELINQUENT    553714
              4.Silver Age          DELINQUENT    297330
              5.Golden Age          DELINQUENT    188222
              1.Young Professional  DELINQUENT     81519
              2.Young Families      DELINQUENT     64970
1             3.Mid Lifers          DELINQUENT     11111
              4.Silver Age          DELINQUENT      6158
              5.Golden Age          DELINQUENT      3996
30            3.Mid Lifers          DELINQUENT      2841
0             0.Younger             DELINQUENT      2427
60            3.Mid Lifers          DELINQUENT      1787
30            4.Silver Age          DELINQUENT      1539
1             1.Young Professional  DELINQUENT      1468
90            3.Mid Lifers          DELINQUENT      1249
1             2.Young Families      DELINQUENT      1239
120           3.Mid Lifers          DELINQUENT       999
30            5.Golden Age          DELIN

## 1. SAGA total monthly bad debt

In [21]:
monthly = pd.pivot_table(MLs, 
                         values=['GCIF_NO','CURR_LMT_AMT','BAL'],
                         index=['BASE_DT'],
                         columns=['CC_STATUS'],
                         aggfunc={'GCIF_NO':'count',
                                  'CURR_LMT_AMT':'sum',
                                  'BAL':'sum'})

In [22]:
print(monthly)

                    BAL  CURR_LMT_AMT    GCIF_NO
CC_STATUS    DELINQUENT    DELINQUENT DELINQUENT
BASE_DT                                         
20231130   1.561560e+12  3.749174e+12     187885
20231229   1.562580e+12  3.775570e+12     186569
20240131   1.603963e+12  3.848475e+12     183281
20240229   1.629879e+12  3.925737e+12     182894
20240430   1.694901e+12  3.977078e+12     180496
20240531   1.699164e+12  4.037320e+12     180056
20240628   1.758796e+12  4.130127e+12     179964
20240731   1.771556e+12  4.370452e+12     177991
20240829   1.802961e+12  4.406276e+12     178580
20240930   1.852019e+12  4.406244e+12     177997
20241031   1.862254e+12  4.486219e+12     179387


1. Hitung total bad debt SAGA all month
2. Bandingkan bad debt SAGA vs all other CC
2. Bandingkan 